# 📘 FILE 1-B – Eager Execution & GradientTape

## 🎯 Mục tiêu

Sau bài này bạn sẽ hiểu:
- **Eager Execution** là gì và tại sao nó quan trọng
- **Gradient** là gì (theo cách intuitively)
- Cách dùng `tf.GradientTape` để tính gradient
- Cách train model **KHÔNG dùng `model.fit()`** (training loop thủ công)
- Implement Linear Regression từ đầu

---

## 📌 Tại sao phải học GradientTape?

- `model.fit()` rất tiện nhưng **che giấu cơ chế bên trong**
- Muốn custom training loop → **bắt buộc dùng GradientTape**
- Hiểu GradientTape → hiểu cách Deep Learning hoạt động
- Production/MLOps thường cần custom logic

---

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")

---

## 1️⃣ Eager Execution là gì?

### 🔹 TensorFlow 1.x (Graph Mode - cũ)

```python
# TensorFlow 1.x
x = tf.placeholder(tf.float32)
y = x * 2
sess = tf.Session()
result = sess.run(y, feed_dict={x: 3.0})
```

**Vấn đề:**
- Phải build graph trước
- Phải dùng Session
- **Khó debug**

---

### 🔹 TensorFlow 2.x (Eager Execution - mặc định)

```python
# TensorFlow 2.x
x = tf.constant(3.0)
y = x * 2
print(y)  # tf.Tensor(6.0, shape=(), dtype=float32)
```

**Lợi ích:**
- Code chạy **ngay lập tức** (như NumPy)
- Dễ debug
- Pythonic hơn

---

In [ ]:
# Kiểm tra Eager Execution
print(f"Eager execution enabled: {tf.executing_eagerly()}")  # True trong TF 2.x

# Ví dụ đơn giản
x = tf.constant([1, 2, 3])
y = x * 2
print(f"x = {x}")
print(f"y = {y}")
print(f"y.numpy() = {y.numpy()}")  # Chuyển về NumPy ngay

---

## 2️⃣ Gradient là gì? (Intuitively)

### 🔹 Định nghĩa đơn giản

**Gradient** = Độ dốc (slope) của hàm số

**Ví dụ:**

```
f(x) = x²
```

- Tại `x = 2`: gradient = `2x = 4`
- Tại `x = -3`: gradient = `2x = -6`

### 🔹 Tại sao cần gradient?

**Gradient Descent** = Thuật toán tối ưu dựa trên gradient

```
Công thức:
w_new = w_old - learning_rate × gradient
```

- Gradient **dương** → giảm `w`
- Gradient **âm** → tăng `w`
- Mục tiêu: **tìm điểm có loss nhỏ nhất**

---

In [ ]:
# Visualization: f(x) = x²
x_vals = np.linspace(-5, 5, 100)
y_vals = x_vals ** 2

plt.figure(figsize=(10, 4))

# Plot hàm số
plt.subplot(1, 2, 1)
plt.plot(x_vals, y_vals, 'b-', linewidth=2)
plt.scatter([2], [4], color='red', s=100, zorder=5)
plt.title('f(x) = x²')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.grid(True, alpha=0.3)

# Plot gradient
plt.subplot(1, 2, 2)
gradient_vals = 2 * x_vals  # df/dx = 2x
plt.plot(x_vals, gradient_vals, 'g-', linewidth=2)
plt.scatter([2], [4], color='red', s=100, zorder=5)
plt.title('Gradient: df/dx = 2x')
plt.xlabel('x')
plt.ylabel('gradient')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

print("Tại x=2: gradient = 4 (dốc lên)")
print("Tại x=-3: gradient = -6 (dốc xuống)")
print("Tại x=0: gradient = 0 (điểm cực tiểu)")

---

## 3️⃣ tf.GradientTape - Cách dùng cơ bản

### 🔹 Cú pháp cơ bản

```python
with tf.GradientTape() as tape:
    # 1. Watch variables (nếu cần)
    tape.watch(x)
    
    # 2. Forward pass
    y = f(x)

# 3. Tính gradient
grad = tape.gradient(y, x)
```

### 🔹 Quy tắc quan trọng

1. **Chỉ watch `tf.Variable`** (mặc định) hoặc `tape.watch()` cho tensor thường
2. **Gradient chỉ tính 1 lần** (tape bị xóa sau `gradient()`)
3. **persistent=True** nếu muốn tính nhiều lần

---

In [ ]:
# Ví dụ 1: Tính gradient của f(x) = x²
x = tf.Variable(2.0)  # tf.Variable tự động được watch

with tf.GradientTape() as tape:
    y = x ** 2  # f(x) = x²

grad = tape.gradient(y, x)
print(f"x = {x.numpy()}")
print(f"y = x² = {y.numpy()}")
print(f"dy/dx = 2x = {grad.numpy()}")
print(f"Kiểm tra: 2 × {x.numpy()} = {2 * x.numpy()}")

In [ ]:
# Ví dụ 2: Watch tensor thường
x = tf.constant(3.0)  # Tensor thường

with tf.GradientTape() as tape:
    tape.watch(x)  # BẮT BUỘC watch nếu không phải Variable
    y = x ** 3  # f(x) = x³

grad = tape.gradient(y, x)
print(f"x = {x.numpy()}")
print(f"y = x³ = {y.numpy()}")
print(f"dy/dx = 3x² = {grad.numpy()}")
print(f"Kiểm tra: 3 × {x.numpy()}² = {3 * x.numpy() ** 2}")

In [ ]:
# Ví dụ 3: Gradient của nhiều biến
w = tf.Variable(2.0)
b = tf.Variable(1.0)
x = tf.constant(3.0)

with tf.GradientTape() as tape:
    y = w * x + b  # y = wx + b

# Tính gradient theo w và b
grad_w, grad_b = tape.gradient(y, [w, b])

print(f"y = {w.numpy()} × {x.numpy()} + {b.numpy()} = {y.numpy()}")
print(f"dy/dw = x = {grad_w.numpy()}")
print(f"dy/db = 1 = {grad_b.numpy()}")

In [ ]:
# Ví dụ 4: persistent=True
x = tf.Variable(2.0)

with tf.GradientTape(persistent=True) as tape:
    y = x ** 2
    z = x ** 3

# Tính nhiều gradient từ cùng 1 tape
grad_y = tape.gradient(y, x)
grad_z = tape.gradient(z, x)

print(f"dy/dx = {grad_y.numpy()}")
print(f"dz/dx = {grad_z.numpy()}")

# QUAN TRỌNG: Xóa tape để giải phóng bộ nhớ
del tape

---

## 4️⃣ Training Loop Thủ Công

### 🔹 Quy trình cơ bản

```python
for epoch in range(num_epochs):
    with tf.GradientTape() as tape:
        # 1. Forward pass
        predictions = model(X)
        loss = loss_function(y_true, predictions)
    
    # 2. Backward pass (tính gradient)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # 3. Update weights
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
```

---

In [ ]:
# Ví dụ đơn giản: Tối ưu f(x) = (x - 3)²
# Mục tiêu: tìm x sao cho f(x) nhỏ nhất (x = 3)

x = tf.Variable(0.0)  # Khởi tạo x = 0
learning_rate = 0.1

history = []

for step in range(50):
    with tf.GradientTape() as tape:
        loss = (x - 3) ** 2  # f(x) = (x - 3)²
    
    # Tính gradient
    grad = tape.gradient(loss, x)
    
    # Update x
    x.assign(x - learning_rate * grad)
    
    history.append((step, x.numpy(), loss.numpy()))
    
    if step % 10 == 0:
        print(f"Step {step:2d}: x = {x.numpy():.4f}, loss = {loss.numpy():.4f}")

print(f"\nKết quả cuối: x = {x.numpy():.4f} (mong muốn: 3.0)")

In [ ]:
# Visualization
steps, x_vals, loss_vals = zip(*history)

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(steps, x_vals, 'b-', linewidth=2)
plt.axhline(y=3, color='r', linestyle='--', label='Target (x=3)')
plt.xlabel('Step')
plt.ylabel('x')
plt.title('x convergence')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(steps, loss_vals, 'g-', linewidth=2)
plt.xlabel('Step')
plt.ylabel('Loss')
plt.title('Loss convergence')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 5️⃣ Linear Regression từ đầu

### 🔹 Bài toán

Cho dữ liệu `(x, y)` thỏa mãn:

```
y = 2x + 1 + noise
```

Tìm `w` và `b` sao cho:

```
y_pred = w * x + b
```

### 🔹 Loss function

**Mean Squared Error (MSE):**

```
loss = mean((y_true - y_pred)²)
```

---

In [ ]:
# Tạo dữ liệu giả
np.random.seed(42)

# True parameters
true_w = 2.0
true_b = 1.0

# Generate data
num_samples = 100
X = np.random.randn(num_samples, 1).astype(np.float32)
y = true_w * X + true_b + np.random.randn(num_samples, 1).astype(np.float32) * 0.5

# Convert to TensorFlow
X = tf.constant(X)
y = tf.constant(y)

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Plot
plt.figure(figsize=(8, 6))
plt.scatter(X.numpy(), y.numpy(), alpha=0.5)
plt.xlabel('X')
plt.ylabel('y')
plt.title('Training data')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Khởi tạo parameters
w = tf.Variable(tf.random.normal([1, 1]), name='weight')
b = tf.Variable(tf.zeros([1]), name='bias')

print(f"Initial w: {w.numpy().flatten()}")
print(f"Initial b: {b.numpy().flatten()}")
print(f"\nTrue values: w={true_w}, b={true_b}")

In [ ]:
# Training loop
learning_rate = 0.1
num_epochs = 100

loss_history = []

for epoch in range(num_epochs):
    with tf.GradientTape() as tape:
        # 1. Forward pass
        y_pred = tf.matmul(X, w) + b
        
        # 2. Compute loss (MSE)
        loss = tf.reduce_mean(tf.square(y - y_pred))
    
    # 3. Backward pass
    gradients = tape.gradient(loss, [w, b])
    
    # 4. Update parameters
    w.assign(w - learning_rate * gradients[0])
    b.assign(b - learning_rate * gradients[1])
    
    loss_history.append(loss.numpy())
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d}: loss = {loss.numpy():.4f}, "
              f"w = {w.numpy().flatten()[0]:.4f}, b = {b.numpy()[0]:.4f}")

print(f"\n{'='*60}")
print(f"Final results:")
print(f"  Learned: w = {w.numpy().flatten()[0]:.4f}, b = {b.numpy()[0]:.4f}")
print(f"  True:    w = {true_w:.4f}, b = {true_b:.4f}")
print(f"  Error:   w = {abs(w.numpy().flatten()[0] - true_w):.4f}, "
      f"b = {abs(b.numpy()[0] - true_b):.4f}")

In [ ]:
# Visualization
plt.figure(figsize=(14, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(loss_history, 'b-', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)

# Plot predictions
plt.subplot(1, 2, 2)
plt.scatter(X.numpy(), y.numpy(), alpha=0.5, label='Data')

# Learned line
x_line = np.linspace(X.numpy().min(), X.numpy().max(), 100).reshape(-1, 1)
y_line = w.numpy() * x_line + b.numpy()
plt.plot(x_line, y_line, 'r-', linewidth=2, label='Learned')

# True line
y_true_line = true_w * x_line + true_b
plt.plot(x_line, y_true_line, 'g--', linewidth=2, label='True')

plt.xlabel('X')
plt.ylabel('y')
plt.title('Linear Regression Result')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 6️⃣ Sử dụng Optimizer

### 🔹 Vấn đề với manual update

```python
w.assign(w - learning_rate * grad)  # Phải tự implement
```

### 🔹 Giải pháp: tf.keras.optimizers

```python
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
optimizer.apply_gradients(zip(gradients, variables))
```

**Lợi ích:**
- Code ngắn gọn hơn
- Dễ thay đổi optimizer (Adam, RMSprop...)
- Support advanced features (momentum, decay...)

---

In [ ]:
# Linear Regression với Optimizer
# Reset parameters
w = tf.Variable(tf.random.normal([1, 1]), name='weight')
b = tf.Variable(tf.zeros([1]), name='bias')

# Tạo optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

loss_history_opt = []

for epoch in range(100):
    with tf.GradientTape() as tape:
        y_pred = tf.matmul(X, w) + b
        loss = tf.reduce_mean(tf.square(y - y_pred))
    
    # Tính gradient
    gradients = tape.gradient(loss, [w, b])
    
    # Update bằng optimizer (NGẮN GỌN HƠN!)
    optimizer.apply_gradients(zip(gradients, [w, b]))
    
    loss_history_opt.append(loss.numpy())
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d}: loss = {loss.numpy():.4f}")

print(f"\nFinal: w = {w.numpy().flatten()[0]:.4f}, b = {b.numpy()[0]:.4f}")

---

## 7️⃣ Common Mistakes

### ❌ Mistake 1: Quên watch tensor thường

In [ ]:
# SAI
x = tf.constant(2.0)

with tf.GradientTape() as tape:
    # QUÊN tape.watch(x)
    y = x ** 2

grad = tape.gradient(y, x)
print(f"Gradient: {grad}")  # None!

In [ ]:
# ĐÚNG
x = tf.constant(2.0)

with tf.GradientTape() as tape:
    tape.watch(x)  # BẮT BUỘC!
    y = x ** 2

grad = tape.gradient(y, x)
print(f"Gradient: {grad.numpy()}")  # 4.0

### ❌ Mistake 2: Dùng tape 2 lần

In [ ]:
# SAI
x = tf.Variable(2.0)

with tf.GradientTape() as tape:
    y = x ** 2

grad1 = tape.gradient(y, x)
print(f"Grad 1: {grad1.numpy()}")

# LỖI: tape đã bị xóa
try:
    grad2 = tape.gradient(y, x)
except RuntimeError as e:
    print(f"\nError: {e}")

In [ ]:
# ĐÚNG
x = tf.Variable(2.0)

with tf.GradientTape(persistent=True) as tape:
    y = x ** 2

grad1 = tape.gradient(y, x)
grad2 = tape.gradient(y, x)
print(f"Grad 1: {grad1.numpy()}")
print(f"Grad 2: {grad2.numpy()}")

del tape  # Nhớ xóa!

### ❌ Mistake 3: Quên convert NumPy → TensorFlow

In [ ]:
# SAI
w = tf.Variable(2.0)
x = np.array([1.0, 2.0, 3.0])  # NumPy array

with tf.GradientTape() as tape:
    # CẢNH BÁO: Mixing NumPy and TensorFlow
    y = w * x  # Hoạt động nhưng không tối ưu

grad = tape.gradient(y, w)
print(f"Gradient: {grad}")

In [ ]:
# ĐÚNG
w = tf.Variable(2.0)
x = tf.constant([1.0, 2.0, 3.0])  # TensorFlow tensor

with tf.GradientTape() as tape:
    y = w * x

grad = tape.gradient(y, w)
print(f"Gradient: {grad.numpy()}")

---

## 8️⃣ Best Practices

### ✅ 1. Luôn dùng tf.Variable cho trainable parameters

```python
# ĐÚNG
w = tf.Variable(initial_value)

# SAI (không trainable)
w = tf.constant(initial_value)
```

### ✅ 2. Dùng optimizer thay vì manual update

```python
# TỐT
optimizer = tf.keras.optimizers.Adam(0.001)
optimizer.apply_gradients(zip(grads, vars))

# TỆ (khó maintain)
w.assign(w - lr * grad)
```

### ✅ 3. Kiểm tra gradient

```python
if grad is None:
    print("WARNING: Gradient is None!")
```

### ✅ 4. Xóa persistent tape

```python
with tf.GradientTape(persistent=True) as tape:
    ...

# Tính gradient
...

# QUAN TRỌNG
del tape
```

---

---

## 9️⃣ Exercises

### 📝 Exercise 1: Gradient của hàm phức tạp

Tính gradient của:

```
f(x) = 3x³ - 2x² + 5x - 1
```

tại `x = 2`

**Hint:** df/dx = 9x² - 4x + 5

In [ ]:
# YOUR CODE HERE
x = tf.Variable(2.0)

with tf.GradientTape() as tape:
    # TODO: Implement f(x) = 3x³ - 2x² + 5x - 1
    pass

# TODO: Compute gradient
# Expected: 9(4) - 4(2) + 5 = 36 - 8 + 5 = 33

### 📝 Exercise 2: Polynomial Regression

Implement polynomial regression:

```
y = w2*x² + w1*x + b
```

Cho dữ liệu:

```python
X = [-2, -1, 0, 1, 2]
y = [6, 2, 0, 0, 2]  # y ≈ x² - x
```

In [ ]:
# YOUR CODE HERE
X_data = tf.constant([[-2.], [-1.], [0.], [1.], [2.]], dtype=tf.float32)
y_data = tf.constant([[6.], [2.], [0.], [0.], [2.]], dtype=tf.float32)

# TODO: Khởi tạo w2, w1, b
# TODO: Training loop
# TODO: Plot kết quả

# Expected: w2 ≈ 1, w1 ≈ -1, b ≈ 0

### 📝 Exercise 3: Logistic Regression (BONUS)

Implement binary classification:

```python
z = w*x + b
y_pred = sigmoid(z)
loss = binary_crossentropy(y_true, y_pred)
```

**Hint:**

```python
sigmoid(z) = 1 / (1 + exp(-z))
binary_crossentropy = -mean(y*log(y_pred) + (1-y)*log(1-y_pred))
```

In [ ]:
# YOUR CODE HERE
# TODO: Generate binary classification data
# TODO: Implement logistic regression
# TODO: Plot decision boundary

---

## 🎯 Tóm tắt

### ✅ Những gì đã học

1. **Eager Execution**: Code chạy ngay lập tức (như NumPy)
2. **Gradient**: Độ dốc của hàm số, dùng để tối ưu
3. **tf.GradientTape**: Tool để tính gradient tự động
4. **Training Loop**: Forward → Backward → Update
5. **Linear Regression**: Implement từ đầu không dùng `model.fit()`

### 🎓 Key Takeaways

```python
# Pattern chuẩn
with tf.GradientTape() as tape:
    predictions = model(X)
    loss = loss_fn(y_true, predictions)

grads = tape.gradient(loss, model.trainable_variables)
optimizer.apply_gradients(zip(grads, model.trainable_variables))
```

### 📚 Next Steps

- **File 1-C**: tf.keras & First Neural Network
- **File 1-D**: GPU, Debugging & Exercises

---

## 📖 References

- [TensorFlow GradientTape Guide](https://www.tensorflow.org/guide/autodiff)
- [TensorFlow Eager Execution](https://www.tensorflow.org/guide/eager)
- [Custom Training Walkthrough](https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough)

---

**Chúc bạn học tốt! 🚀**